<a href="https://colab.research.google.com/github/nrasadi/API-Load-Testing-Tool/blob/master/cifar10_mobilenetV3small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!nvidia-smi

Wed Feb  3 18:29:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone https://github.com/Scalable-DNN/pretrained-models.git

Cloning into 'pretrained-models'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [2]:
!pip install larq

     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=42affdeb4873ca936c8b9ccb9487e21cc8e2f7d9d7654b3f51cc238f4adc65a5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [3]:
# %%bash
# cd pretrained-models
# git pull https://github.com/Scalable-DNN/pretrained-models.git

In [6]:
import os, sys
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import applications as kapp, models
from tensorflow.keras.datasets import cifar10
import larq as lq

import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [9]:
print("X_train.shape: {}\tY_train.shape: {}".format(x_train.shape, y_train.shape))
print("X_val.shape:   {}\tY_val.shape:   {}".format(x_val.shape, y_val.shape))
print("X_test.shape:  {}\tY_test.shape:  {}".format(x_test.shape, y_test.shape))

X_train.shape: (40000, 32, 32, 3)	Y_train.shape: (40000, 1)
X_val.shape:   (10000, 32, 32, 3)	Y_val.shape:   (10000, 1)
X_test.shape:  (10000, 32, 32, 3)	Y_test.shape:  (10000, 1)


In [10]:
to_categorical = keras.utils.to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [65]:
DataGen = keras.preprocessing.image.ImageDataGenerator

transformations = {
    'rotation_range': 5,
    # 'featurewise_std_normalization': True,
    # 'featurewise_center': True,
    'zoom_range': .2,
    'horizontal_flip': True,
    # 'preprocessing_function': kapp.mobilenet_v3.preprocess_input
}

train_generator = DataGen(**transformations)
val_generator   = DataGen(**transformations)
test_generator  = DataGen(**transformations)

In [66]:
train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [72]:
core_model = kapp.MobileNetV3Small(x_train.shape[1:], weights='imagenet', include_top=False, classes=y_train.shape[-1])
core_model.trainable = True

inputs = keras.layers.Input(shape=x_train.shape[1:])

norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(inputs)
norm_layer.set_weights([mean, var])

x = core_model(x, training=True)
x = keras.layers.GlobalAveragePooling2D(name='global_average_pooling')(x)
x = keras.layers.Reshape((1, 1, -1))(x)
x = keras.layers.Dropout(.2, name='dropout', )(x)
x = keras.layers.Conv2D(filters = y_train.shape[-1], kernel_size=1, name='logits')(x)
x = keras.layers.Flatten()(x)
outputs = keras.layers.Activation('softmax', name='predictions')(x)

model = models.Model(inputs=inputs, outputs=outputs)

# model.summary()
# lq.models.summary(model)

In [73]:
batch_size = 8192
epochs = 1000
lr = 1e-2

optimzer = keras.optimizers.Adam(learning_rate=lr)
callbacks = [
            keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=.1, patience=5),
            keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/0Works/cooparetive/mobilenetV3', monitor='val_accuracy', save_best_only=True)
]



## Transfer Learning

In [74]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(core_model.layers))

core_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in core_model.layers[:fine_tune_at]:
  layer.trainable =  False


model.compile(optimizer=optimzer, loss='categorical_crossentropy', metrics=['accuracy'], )

Number of layers in the base model:  243


In [70]:
# model.summary()

In [75]:
model.fit(
    train_generator.flow(x_train, y_train, batch_size),
    epochs=epochs,
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=val_generator.flow(x_val, y_val, batch_size),
    validation_steps=x_val.shape[0] // batch_size,
    callbacks=callbacks,
)

KeyboardInterrupt: ignored

## Fine-Tuning

In [25]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(core_model.layers))

core_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in core_model.layers[:fine_tune_at]:
  layer.trainable =  False


Number of layers in the base model:  243


In [26]:
epochs = 500
model.compile(optimizer=optimzer, loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'], )

In [ ]:
model.fit(
    train_generator.flow(x_train, y_train, batch_size),
    epochs=epochs,
    steps_per_epoch=x_train.shape[0] // batch_size,
    validation_data=val_generator.flow(x_val, y_val, batch_size),
    validation_steps=x_val.shape[0] // batch_size,
    callbacks=callbacks,
)

In [96]:
%%bash
cd pretrained-models/
mkdir mobilenetV3small
cp "/content/drive/MyDrive/Colab\ Notebooks/cocifar10-mobilenetV3small.ipynb" "/content/pretrained-models/mobilenetV3small"

mkdir: cannot create directory ‘mobilenetV3small’: File exists
cp: cannot stat '/content/drive/MyDrive/Colab\ Notebooks/cocifar10-mobilenetV3small.ipynb': No such file or directory


In [101]:
!cp "/content/drive/MyDrive/Colab Notebooks/cifar10-mobilenetV3small.ipynb" "/content/pretrained-models/mobilenetV3small"

In [105]:
%%bash
cd "/content/pretrained-models/"
# git add -a
# git commit -m "Add mobilenetV3small notebook"
# git status
# git push nrasadi@github.com master
git config --global 

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	mobilenetV3small/

nothing added to commit but untracked files present (use "git add" to track)



*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@65611df569ed.(none)')
error: src refspec master does not match any.
error: failed to push some refs to 'nrasadi@github.com'


In [89]:
%%bash 
echo $PWD

/content


In [94]:
!ls drive/MyDrive/Colab\ Notebooks

 7segment-02.ipynb
 7segment.ipynb
'BBox Proposal Based on Edge Detection.ipynb'
 charnet.ipynb
 CIFAR10-BNN.ipynb
'cifar10-mobilenetV3small (1).ipynb'
 cifar10-mobilenetV3small.ipynb
'Copy of 2.5 Audio Recognition using Siamese Network.ipynb'
'Copy of 7segment.ipynb'
'Copy of cifar10-mobilenetV3small.ipynb'
'Copy of Course 1 - Part 4 - Lesson 2 - Notebook.ipynb'
'Copy of demo.ipynb'
'Copy of Exercise 1 - House Prices - Question.ipynb'
'Copy of Exercise2-Question.ipynb'
'Copy of interactive_eager_few_shot_od_training_colab.ipynb'
'Copy of Mask-Navidreza (1).ipynb'
'Copy of Mask-Navidreza.ipynb'
'Copy of Mask-Navidreza-split.ipynb'
'Copy of Mask R-CNN Demo'
'Copy of Object detection'
'Copy of object_detection_tutorial.ipynb'
'Copy of Roboflow-TensorFlow2-Object-Detection.ipynb'
'Data Augmentation.ipynb'
'download kitti.ipynb'
 EfficientDNN-HW1.ipynb
'EfficientNet Keras Transfer Learning.ipynb'
'evaluate imagenet validation set.ipynb'
 generate_glucometers.ipynb
'GettingFamilliarWith Pas